In [35]:
import matplotlib.pyplot as plt
import numpy as np
import os
import folium
import pandas as pd

In [5]:
def check_path_exists(path):
    print(os.path.exists(path))

In [ ]:
def load_geolife_plt(filepath: str) -> pd.DataFrame:
    df = pd.read_csv(filepath, skiprows=6, header=None)
    df.columns = ["lat", "lon", "unused", "alt", "timestamp", "date", "time"]
    return df[["lat", "lon", "alt", "date", "time"]]


In [104]:
# decode hash to obtain min and max lat lon for each geohash
def decode_geohash_mine(geohash: str) -> tuple:
    def refine_interval(interval, char_index, mask):
        if char_index & mask:
            interval[0] = (interval[0] + interval[1]) / 2
        else:
            interval[1] = (interval[0] + interval[1]) / 2
        return interval
    _base32 = '0123456789bcdefghjkmnpqrstuvwxyz'
    _bits = [16, 8, 4, 2, 1]
    lat_interval = [-90.0, 90.0]
    lon_interval = [-180.0, 180.0]
    is_lon = True
    for char in geohash:
        char_index = _base32.index(char)
        for mask in _bits:
            if is_lon:
                lon_interval = refine_interval(lon_interval, char_index, mask)
            else:
                lat_interval = refine_interval(lat_interval, char_index, mask)
            is_lon = not is_lon
    return (lat_interval[0], lat_interval[1], lon_interval[0], lon_interval[1])

In [116]:
precision = [6,7,8]
p = precision[2]
trajectory_id = "001_0"

In [117]:
PATH_TRAJECTORIES = r"D:/Topicos en manejo de grandes volumenes de datos/Proyecto/output_renombrado"
PATH_TRAJECTORIES_GEOHASH = f"D:/Topicos en manejo de grandes volumenes de datos/Proyecto/output_geohash_unique/precision_{p}"
check_path_exists(PATH_TRAJECTORIES)
check_path_exists(PATH_TRAJECTORIES_GEOHASH)

format_plt = ".plt"
format_csv = ".csv"
FINAL_PATH_TRAYECTORY = PATH_TRAJECTORIES + "/" +trajectory_id + format_plt
print(FINAL_PATH_TRAYECTORY)

FINAL_PATH_TRAYECTORY_GEOHASH = PATH_TRAJECTORIES_GEOHASH + "/" +trajectory_id + format_csv
print(FINAL_PATH_TRAYECTORY_GEOHASH)

check_path_exists(FINAL_PATH_TRAYECTORY)
check_path_exists(FINAL_PATH_TRAYECTORY_GEOHASH)

df = load_geolife_plt(FINAL_PATH_TRAYECTORY)
print(df.head())
df_2 = pd.read_csv(FINAL_PATH_TRAYECTORY_GEOHASH)
print(df_2.head())
print(df_2.shape)

True
True
D:/Topicos en manejo de grandes volumenes de datos/Proyecto/output_renombrado/001_0.plt
D:/Topicos en manejo de grandes volumenes de datos/Proyecto/output_geohash_unique/precision_8/001_0.csv
True
True
         lat         lon  alt        date      time
0  39.984094  116.319236  492  2008-10-23  05:53:05
1  39.984198  116.319322  492  2008-10-23  05:53:06
2  39.984224  116.319402  492  2008-10-23  05:53:11
3  39.984211  116.319389  492  2008-10-23  05:53:16
4  39.984217  116.319422  491  2008-10-23  05:53:21
         lat         lon  alt        date      time   geohash
0  39.984094  116.319236  492  2008-10-23  05:53:05  wx4eqyvh
1  39.984710  116.319865  320  2008-10-23  05:53:23  wx4eqyvx
2  39.984623  116.319773  326  2008-10-23  05:53:33  wx4eqyvr
3  39.984555  116.319728  324  2008-10-23  05:53:43  wx4eqyvq
4  39.984959  116.319969  304  2008-10-23  05:54:03  wx4eqzj9
(374, 6)


In [ ]:
map_center = [(df['lat'].min() + df['lat'].max()) / 2, (df['lon'].min() + df['lon'].max()) / 2]
m = folium.Map(location=map_center, zoom_start=12, tiles="Cartodb positron")
# plot trajectory
folium.PolyLine(df[['lat','lon']].values.tolist(), color='blue').add_to(m)

# dibujar las celdas usadas
for g in df_2['geohash'].unique():
    lat_min, lat_max, lon_min, lon_max = decode_geohash_mine(g)
    folium.Rectangle(
        bounds=[[lat_min, lon_min], [lat_max, lon_max]],
        color='red',
        fill=True,
        fill_opacity=0.2
    ).add_to(m)
m.save(f"trajectory_geohash_map_polyline_{trajectory_id}_{p}.html")

In [ ]:
map_center = [(df['lat'].min() + df['lat'].max()) / 2,
              (df['lon'].min() + df['lon'].max()) / 2]

m = folium.Map(location=map_center, zoom_start=12, tiles="Cartodb positron")
-
for lat, lon in df[['lat', 'lon']].values:
    folium.CircleMarker(
        location=[lat, lon],
        radius=2,
        color="blue",
        fill=True,
        fill_opacity=0.8
    ).add_to(m)

for g in df_2['geohash'].unique():
    lat_min, lat_max, lon_min, lon_max = decode_geohash_mine(g)
    folium.Rectangle(
        bounds=[[lat_min, lon_min], [lat_max, lon_max]],
        color='red',
        fill=True,
        fill_opacity=0.2
    ).add_to(m)

m.save(f"trajectory_geohash_map{trajectory_id}_{p}.html")
